In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


# Lexical / n-gram overlap metrics

## BLEU

In [7]:
import evaluate
bleu = evaluate.load('bleu')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
predictions = ["I have thirty six years"]

references = [
    [["I am thirty six years old"],["I am thirty six"]]
]

bleu.compute(predictions=predictions,references=references)

# bleu : geometric mean of all 4 n-gram precisions
# precisions : unigram,bigram,trigram and 4-gram precision scores

# Cons of bleu metrics
# -> struggles with non_english languages
# -> hard to compare scores with different tokenizers

{'bleu': 0.0,
 'precisions': [0.6, 0.5, 0.3333333333333333, 0.0],
 'brevity_penalty': 0.8187307530779819,
 'length_ratio': 0.8333333333333334,
 'translation_length': 5,
 'reference_length': 6}

## SacreBLEU

In [15]:
# use sacreBLEU metrics
# address the tokenization limitations of bleu
!pip install sacrebleu
sacrebleu = evaluate.load('sacrebleu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.8 MB/s eta 0:00:00


In [16]:
predictions = ["I have thirty six years"]

references = [
    ["I am thirty six years old","I am thirty six"]
]
sacrebleu.compute(predictions=predictions,references=references)

{'score': 42.7287006396234,
 'counts': [4, 2, 1, 0],
 'totals': [5, 4, 3, 2],
 'precisions': [80.0, 50.0, 33.333333333333336, 25.0],
 'bp': 1.0,
 'sys_len': 5,
 'ref_len': 4}

## ROUGE

In [ ]:
# metric for text summarization
# ROUGE compares a generated summary to one or more ref summaries
# ROUGE-N compare n-grams of the generation with n-grams of the references

In [17]:
!pip install nltk rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=cc2cf05d8526efbb98f40db631a94fa8ab921ca384a0a01e8cb721deb86d5358
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [18]:
rouge = evaluate.load('rouge')
preditions = ["I really loved reading the Hunger Games"]
refrences = ["I loved reading the Hunger Games"]
rouge.compute(predictions=predictions,references=refrences)

{'rouge1': np.float64(0.1818181818181818),
 'rouge2': np.float64(0.0),
 'rougeL': np.float64(0.1818181818181818),
 'rougeLsum': np.float64(0.1818181818181818)}

# Semantic Similarity Metrics


These focus on meaning, not exact wording.

In [20]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00


## BERT


In [25]:
from bert_score import score
generated_text = ["The quick brown fox jumps over the lazy dog"]
original_text = ["A fast brown fox leaps ovr a sleepy dog"]

P,R,F1 = score(generated_text,original_text,lang = "en",verbose=True)
print(P,R,F1)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.05 seconds, 0.96 sentences/sec
tensor(0.9596) tensor([0.9288]) tensor([0.9439])


In [26]:
generated_text = ["AI is future of technology"]
original_text = ["A fast brown fox leaps ovr a sleepy dog"]

P,R,F1 = score(generated_text,original_text,lang = "en",verbose=True)
print(P,R,F1)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.51 seconds, 0.40 sentences/sec
tensor([0.8274]) tensor([0.8081]) tensor([0.8176])


## SBERT

In [27]:
!pip install sentence-transformers

In [39]:
# using sbert
from sentence_transformers import SentenceTransformer           # loads pre-trained sbert models
from sklearn.metrics.pairwise import cosine_similarity          # for comparing two vecrors


# all-MiniLM-L6-v2 is a pretrained sentence-embedding model that converts a full sentence into a single numerical vector that represents its meaning.
model = SentenceTransformer("all-MiniLM-L6-v2")

s1 = "I am thirty six years old"
s2 = "My age is 36"


# takes list of sentences
embeddings = model.encode([s1,s2])
# print(embeddings[0].shape)
# print(embeddings[1].shape)
# print(embeddings.shape)

similarity = cosine_similarity(
    [embeddings[0]],
    [embeddings[1]]
)

print(similarity)

(2, 384)
[[0.7777139]]


In [42]:
# using universal sentence encoder
import tensorflow_hub as hub    # loads pretrained models
import tensorflow as tf         # handles tensors and math

model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
sentences = [
    "I am thirty six years old",
    "My age is 36"
]

embeddings = model(sentences)
# print(embeddings.shape)

similarity = tf.keras.losses.cosine_similarity(
    embeddings[0],embeddings[1]
)

# returns negative cosine similarity so we subtract from 1
print(-similarity.numpy())

0.6007238
